### Ideias

* Podemos ter uma segunda avaliação com peso de acordo com os trendings e seus direcionamentos para sentimento... ele pode aumentar ou diminuir o peso do sentimento...
* Vamos usar comitês para tentar melhorar?
* Trending tentaremos obter palavras dos ngrams? E incluí-las? Além do tratamento de trending e sentimento
* We highly recommend the twokenize tokenizer from Allen Ritter’s work (https://github.com/aritter/twitter_nlp/blob/master/python/twokenize.py)

In [94]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import TweetTokenizer, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re,string,unicodedata
from collections import Counter

import matplotlib.cm as cm
import matplotlib.pyplot as plt

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import keras
from keras.models import Sequential
from keras.layers import Dense,Conv1D,MaxPooling1D
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.models import model_from_json
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

import seaborn as sns

from wordsegment import load, segment

import contractions 

## Inicializações

#### NLTK

In [ ]:
nltk.download()
nltk.download('stopwords')

lista_stopwords = set(stopwords.words('english'))
punctuation = list(string.punctuation)
lista_stopwords.update(punctuation)

#word segmentation
load()

In [ ]:
ARQUIVO_GLOVE_100 = 'glove.6B.100d.txt'
ARQUIVO_GLOVE_100_TWITTER = 'glove.twitter.27B.100d.txt'

## Funções

#### Utilitários

In [ ]:
# Carrega o arquivo de Embeddings
def read_embedding(file_name):
    with open(file_name,'r', encoding="utf8") as f:
        word_vocab = {} 
        word2vector = []
        pos = 0
        for line in f:
            line_ = line.strip() 
            words_vec = line_.split()
            
            if (len(words_vec) == 101):
                word_vocab[words_vec[0]] = pos
                word2vector.append(np.array(words_vec[1:],dtype=float)) 
                pos+=1
            
    word2vector = np.stack(word2vector)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector

#### Limpeza dos Dados

In [119]:
def limpa_texto(texto, use_hashtag_embedding):
    # link
    texto = re.sub('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '<url>', texto)
    # numero string com números não estamos retirando inicialmente (COVID-19)
    texto = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', '<number>', texto)
    texto = re.sub(r'\s+', ' ', texto)
    
    #texto = re.sub(r'\b([A-Z]+)\b', '<allcaps> \g<1>', texto)
    
    if use_hashtag_embedding:
        texto = re.sub(r'#([\w]+)', '<hashtag> \g<1>', texto)
    else:
        texto = re.sub(r'#', '', texto)
    return texto

In [ ]:
#Labels. Pemite agrupar negativos e positivos
def get_labels_dict(agrupar_labels):
    if not agrupar_labels:
        return {'Extremely Negative': -2 , 'Negative': -1, 'Neutral': 0, 'Positive': 1, 'Extremely Positive': 2}
    else:
        return {'Negative': -1 ,'Neutral': 0, 'Positive': 1}    
    
    
# Converter labels da representação textual para one hot encode
def convert_labels_to_hot_encode(labels, labels_dict, agrupar_labels):
    if agrupar_labels:
        labels_conv =[labels_dict[label.split()[-1]] for label in labels]
    else:
        labels_conv =[labels_dict[label] for label in labels]
    df_dummified = pd.get_dummies(labels_conv)
    
    columns = df_dummified.columns
    print(columns)
    
    return df_dummified.to_numpy()

---

## Pré-Processamento

### Rotina Pré-Processamento (Representação Vetorial/Contexto)

In [136]:
# separa tokens por representação vetorial
def split_tokens_word_vec(texto, vocab, use_twitter_tokenizer, min_length_word):
    lemmatizer = WordNetLemmatizer()
    
    texto = contractions.fix(texto) 
        
    lista_tokens_inicial = []
    if use_twitter_tokenizer:
        t_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
        lista_tokens_inicial = t_tokenizer.tokenize(texto)
    else:
        lista_tokens_inicial = word_tokenize(texto)
    
    tokens_encontrados = []
    tokens_nao_encontrados = []
    
    for token in lista_tokens_inicial:
        add_token = True
        
        #if token in lista_stopwords:
        #    add_token = False
        
        if len(token) < min_length_word:
            add_token = False
        
        if add_token:
            if token.lower() in vocab:
                tokens_encontrados.append(token.lower())
            else:
                sinonimo = get_sinonimo(token.lower())
                if sinonimo != None:
                    tokens_encontrados.append(sinonimo)
                else:
                    tokens_salvos = trata_token_nao_encontrado(token, vocab)
                    if len(tokens_salvos) > 0:
                        tokens_encontrados.extend(tokens_salvos)
                    else:
                        #tokens_encontrados.append('<unknown>')
                        tokens_nao_encontrados.append(token)
    return tokens_encontrados, tokens_nao_encontrados
        

In [ ]:
def trata_token_nao_encontrado(token, vocab):
    lemmatizer = WordNetLemmatizer()
    tokens_salvos = []
    
    token_lemmatizado = lemmatizer.lemmatize(token)
    
    if token != token_lemmatizado:
        if token_lemmatizado in vocab:
            return [token_lemmatizado]
    
    tokens_word_segmentation = segment(token)
    
    for subtoken in tokens_word_segmentation:
        if subtoken in vocab:
            tokens_salvos.append(subtoken)
        else:
            subtoken_lemmatizado = lemmatizer.lemmatize(subtoken)
            if (subtoken_lemmatizado in vocab):
                tokens_salvos.append(subtoken_lemmatizado)
            else:
                sinonimo = get_sinonimo(subtoken)
                if sinonimo != None:
                    tokens_salvos.append(sinonimo)                
    return tokens_salvos
        
        

In [ ]:
def get_sinonimo(str):
    dict_sinonimos = {'covid': 'coronavirus', 'chloroquine':'medicine'}
    
    sinonimo = None
    
    if str in dict_sinonimos:
        sinonimo = dict_sinonimos[str]
    return sinonimo
        

In [ ]:
def carrega_tokens_word_vec(lista_tweet, vocab, use_twitter_tokenizer,  min_length_word, use_hashtag_embedding):
    feature_tokens = []
    feature_tokens_verificar =[]
    for tweet in lista_tweet:
        texto_limpo = limpa_texto(tweet, use_hashtag_embedding)
        tokens_add, tokens_verify = split_tokens_word_vec(texto_limpo, 
                                                          vocab,
                                                          use_twitter_tokenizer=use_twitter_tokenizer, 
                                                          min_length_word=min_length_word)
        feature_tokens.append(np.asarray(tokens_add))
        feature_tokens_verificar.append(tokens_verify)
    
    
    feature_tokens = np.asarray(feature_tokens)
    feature_tokens_verificar = np.asarray(feature_tokens_verificar)
    
    return feature_tokens, feature_tokens_verificar

In [ ]:
def pre_processamento_word_vec(dados, 
                               agrupar_labels=False, 
                               glove_twitter=True, 
                               min_length_word=0, 
                               convert_labels_hot_encode=True, 
                               use_twitter_tokenizer=True,
                               use_hashtag_embedding=False):
    arquivo_glove = ARQUIVO_GLOVE_100_TWITTER
    if not glove_twitter :
        arquivo_glove = ARQUIVO_GLOVE_100
    
    lista_tweet = dados.OriginalTweet
    labels_str = dados.Sentiment
    lista_sentimentos = dados.Sentiment.unique()
    
    vocab, embedding = read_embedding(arquivo_glove)
    
    labels_dict = get_labels_dict(agrupar_labels)
    if convert_labels_hot_encode:
        labels = convert_labels_to_hot_encode(labels_str, labels_dict, agrupar_labels)
    else:
        if (agrupar_labels):
            labels = [labels_dict[label.split()[-1]] for label in labels_str]
        else:
            labels = [labels_dict[label] for label in labels_str]
        labels = np.asarray(labels)
    
    feature_tokens, feature_tokens_verificar = carrega_tokens_word_vec(lista_tweet, 
                                                                       vocab, 
                                                                       use_twitter_tokenizer=use_twitter_tokenizer, 
                                                                       min_length_word=min_length_word,
                                                                      use_hashtag_embedding=use_hashtag_embedding)
        
    return (vocab, embedding), (feature_tokens, feature_tokens_verificar), labels, labels_dict
    
    
    
def pre_processamento_word_vec_test(dados, 
                                    vocab,
                                    labels_dict,
                                   agrupar_labels=False, 
                                   glove_twitter=True, 
                                   min_length_word=0, 
                                   use_twitter_tokenizer=True, 
                                   use_hashtag_embedding=False):
    
    arquivo_glove = ARQUIVO_GLOVE_100_TWITTER
    if not glove_twitter :
        arquivo_glove = ARQUIVO_GLOVE_100
    
    lista_tweet = dados.OriginalTweet
    labels_str = dados.Sentiment
    
    
    labels = convert_labels_to_hot_encode(labels_str, labels_dict, agrupar_labels)
    
    feature_tokens, _ = carrega_tokens_word_vec(lista_tweet, 
                                                vocab, 
                                                use_twitter_tokenizer=use_twitter_tokenizer, 
                                                min_length_word=min_length_word,
                                                use_hashtag_embedding=use_hashtag_embedding)
        
    return feature_tokens, labels

## Análise dos Dados

In [ ]:
print(len(dados))
print(dados.shape)

In [ ]:
dados.Sentiment.value_counts()

In [ ]:
dados.value_counts(['Location','Sentiment'])

In [ ]:
dados.isnull().sum()

In [ ]:
dados[['OriginalTweet','Sentiment']].head(10)

## Word Cloud

In [ ]:
# Wordcloud
def monta_wordcloud(lista_topicos, texto_ou_frequencia, generate_from_text=True):
    num_topicos = len(lista_topicos)
    ncols = 2
    col = 0
    lin  = 1

    fig, ax = plt.subplots(1, ncols, figsize=(30, 20))

    for topico in lista_topicos:
        if generate_from_text:
            wordcloud = WordCloud(background_color="black",collocations=False,
                              colormap="Oranges_r",
                              width = 1000,
                              height = 1000,
                              max_font_size=1000,
                              max_words=30
                             ).generate(texto_ou_frequencia[topico])
        else:
            wordcloud = WordCloud(background_color="black",collocations=False,
                              colormap="Oranges_r",
                              width = 1000,
                              height = 1000,
                              max_font_size=1000,
                              max_words=30
                             ).generate_from_frequencies(texto_ou_frequencia[topico])
    
        ax[col].imshow(wordcloud) 
        ax[col].set_title(f"{topico}")
        # No axis details
        ax[col].axis("off");
        col = col + 1
        if col >= ncols:
            plt.show()
            lin = lin + 1
            col = 0
            ax_cols =  ncols
            fig, ax = plt.subplots(1, ax_cols, figsize=(30, 20))   

In [ ]:
dados = pd.read_csv('Corona_NLP_train.csv', engine="python")
(_, _), (feature_tokens, feature_tokens_verificar) , labels, labels_dict = pre_processamento_word_vec(dados, glove_twitter=True, convert_labels_hot_encode=False,min_length_word=2)

### Word Cloud por Sentimento

In [ ]:
text_sentimento = {}
for sentimento in labels_dict:
    text_sentimento[sentimento] = " ".join([token for index, f_token in enumerate(feature_tokens) if labels[index] == labels_dict[sentimento] for token in f_token ])  
monta_wordcloud(labels_dict, text_sentimento)

### Word Cloud por Trending/Sentimento

In [ ]:
_count_trends = Counter()
_count_trends.update([token for index, lista_token in enumerate(X_tokens + X_tokens_verificar) if y[index] == sentimento for token in lista_token if re.match(r'#\w+',token) ])
monta_wordcloud(lista_sentimentos, _count_trends, generate_from_text=False)


### Word Cloud palavras não Encontradas (Sem Trending)

In [ ]:
text_sentimento = {}
for sentimento in labels_dict:
    text_sentimento[sentimento] = " ".join([token for index, f_token in enumerate(feature_tokens_verificar) if labels[index] == labels_dict[sentimento] for token in f_token ])  
monta_wordcloud(labels_dict, text_sentimento)



## Modelos

### LSTM

In [132]:
def create_model_lstm_pure(weight_matrix, dim_embedding, max_words_length, num_labels=5):
    model = Sequential()
    model.add(Embedding(len(weight_matrix), dim_embedding, weights=[weight_matrix], input_length=max_words_length, trainable=False))
    # model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    #model.add(Dense(512, activation='relu'))
    #model.add(Dropout(0.50))
    model.add(Dense(num_labels, activation='softmax'))
    # Adam Optimiser
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    
    return model


def create_model_lstm_pure_bi(weight_matrix, dim_embedding, max_words_length, num_labels=5):
    model = Sequential()
    model.add(Embedding(len(weight_matrix), dim_embedding, weights=[weight_matrix], input_length=max_words_length, trainable=False))
    # model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    #model.add(Dense(512, activation='relu'))
    #model.add(Dropout(0.50))
    model.add(Dense(num_labels, activation='softmax'))
    # Adam Optimiser
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    
    return model


def create_model_lstm_dense(weight_matrix, dim_embedding, max_words_length, num_labels=5):
    model = Sequential()
    model.add(Embedding(len(weight_matrix), dim_embedding, weights=[weight_matrix], input_length=max_words_length, trainable=False))
    # model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.30))
    model.add(Dense(num_labels, activation='softmax'))
    # Adam Optimiser
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    
    return model

def create_model_lstm_bi_dense(weight_matrix, dim_embedding, max_words_length, num_labels=5):
    model = Sequential()
    model.add(Embedding(len(weight_matrix), dim_embedding, weights=[weight_matrix], input_length=max_words_length, trainable=False))
    # model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.30))
    model.add(Dense(num_labels, activation='softmax'))
    # Adam Optimiser
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    
    return model

def create_model_lstm_conv(weight_matrix, dim_embedding, max_words_length, num_labels=5):
    model = Sequential()
    model.add(Embedding(len(weight_matrix), dim_embedding, weights=[weight_matrix], input_length=max_words_length, trainable=False))
    # model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(num_labels, activation='softmax'))
    # Adam Optimiser
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    
    return model

def crate_model_double_lstm(weight_matrix, dim_embedding, max_words_length, num_labels=5):
    model = Sequential()
    model.add(Embedding(len(weight_matrix), dim_embedding, weights=[weight_matrix], input_length=max_words_length, trainable=False))
    model.add(Bidirectional(LSTM(120, return_sequences=True, recurrent_dropout=0.5)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(120, recurrent_dropout=0.5)))
    model.add(Dense(60, activation='relu')) #60
    model.add(Dense(num_labels, activation='softmax'))
    # Adam Optimiser 
    # testar Adam(lr=0.01, decay=0.001)
    model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(lr=0.01, decay=0.001), metrics=['accuracy'])
    
    return model

# Execução

In [ ]:
# Converter tokens para a representação word vec
def convert_feature_tokens_to_word_vec(feature_tokens,  vocab, dimension_size=100, max_word_length=50):
    feature_word_vec = np.zeros((len(feature_tokens), max_word_length), dtype='int32')
    
    for c_feature, f_token in enumerate(feature_tokens):
        for c_token,token in enumerate(f_token):
            if(c_token < max_word_length):
                feature_word_vec[c_feature,c_token] = vocab[token]
        
    return feature_word_vec
    

In [140]:
max_word_length=50
use_hashtag_embedding=True


dados = pd.read_csv('Corona_NLP_train.csv', engine="python")
dados_test = pd.read_csv('Corona_NLP_test.csv', engine="python")

(vocab, embedding), (feature_tokens, _), labels, labels_dict = pre_processamento_word_vec(dados, 
                                                                                          agrupar_labels=True,
                                                                                          glove_twitter=True, 
                                                                                          convert_labels_hot_encode=True,
                                                                                          min_length_word=1,
                                                                                          use_twitter_tokenizer=True,
                                                                                          use_hashtag_embedding=use_hashtag_embedding)

X_train, X_val, y_train, y_val = train_test_split(feature_tokens, labels,  test_size=0.2, random_state=0)

X_train = convert_feature_tokens_to_word_vec(X_train, 
                                             vocab,
                                             max_word_length=max_word_length)
X_val = convert_feature_tokens_to_word_vec(X_val, 
                                           vocab, 
                                           max_word_length=max_word_length)

X_test, y_test = pre_processamento_word_vec_test(dados_test,
                                          vocab,
                                          labels_dict,
                                          agrupar_labels=True,
                                          glove_twitter=True,
                                          min_length_word=1,
                                          use_twitter_tokenizer=True,
                                          use_hashtag_embedding=use_hashtag_embedding)

X_test = convert_feature_tokens_to_word_vec(X_test, 
                                           vocab, 
                                           max_word_length=max_word_length)



Total Words in DataSet: 1193513
Int64Index([-1, 0, 1], dtype='int64')
Int64Index([-1, 0, 1], dtype='int64')


In [ ]:
model_pure = create_model_lstm_pure(embedding, 100, max_word_length, len(labels_dict))
history_pure = model_pure.fit(X_train, y_train, validation_data=(X_val, y_val),  batch_size=32, epochs=20)

In [ ]:
evaluate_pure = model_pure.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(evaluate_pure[0],evaluate_pure[1]))

In [ ]:
model_pure_bi = create_model_lstm_pure_bi(embedding, 100, max_word_length, len(labels_dict))
history_pure_bi = model_pure_bi.fit(X_train, y_train, validation_data=(X_val, y_val),  batch_size=32, epochs=20)

In [ ]:
evaluate_pure_bi = model_pure_bi.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(evaluate_pure_bi[0],evaluate_pure_bi[1]))

In [ ]:
model_dense = create_model_lstm_dense(embedding, 100, max_word_length, len(labels_dict))
history_dense = model_dense.fit(X_train, y_train, validation_data=(X_val, y_val),  batch_size=32, epochs=20)

In [ ]:
evaluate_dense = model_dense.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(evaluate_dense[0],evaluate_dense[1]))

In [ ]:
model_bi_dense = create_model_lstm_bi_dense(embedding, 100, max_word_length, len(labels_dict))
history_bi_dense = model_bi_dense.fit(X_train, y_train, validation_data=(X_val, y_val),  batch_size=32, epochs=20)

In [ ]:
evaluate_bi_dense = model_bi_dense.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(evaluate_bi_dense[0],evaluate_bi_dense[1]))

In [ ]:
model_lstm_conv = create_model_lstm_conv(embedding, 100, max_word_length, len(labels_dict))
filepath="weights_best_lstm_cnn.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
callbacks_list = [checkpoint]
history_bi_dense = model_lstm_conv.fit(X_train, y_train, validation_data=(X_val, y_val),  batch_size=32, epochs=20, callbacks=callbacks_list)


In [141]:
model_dobule_lstm = crate_model_double_lstm(embedding, 100, max_word_length, len(labels_dict))

filepath="weights_best_double_lstm.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
callbacks_list = [checkpoint]

history_double_lstm = model_dobule_lstm.fit(X_train, y_train, validation_data=(X_val, y_val),  batch_size=128, epochs=12, callbacks=callbacks_list)



Epoch 1/12
258/258 [==============================] - ETA: 0s - loss: 0.8576 - accuracy: 0.6016
Epoch 00001: val_accuracy improved from -inf to 0.72449, saving model to weights_best_double_lstm.hdf5
258/258 [==============================] - 379s 1s/step - loss: 0.8576 - accuracy: 0.6016 - val_loss: 0.6634 - val_accuracy: 0.7245
Epoch 2/12
258/258 [==============================] - ETA: 0s - loss: 0.6033 - accuracy: 0.7570
Epoch 00002: val_accuracy improved from 0.72449 to 0.78790, saving model to weights_best_double_lstm.hdf5
258/258 [==============================] - 389s 2s/step - loss: 0.6033 - accuracy: 0.7570 - val_loss: 0.5446 - val_accuracy: 0.7879
Epoch 3/12
258/258 [==============================] - ETA: 0s - loss: 0.5021 - accuracy: 0.8087
Epoch 00003: val_accuracy improved from 0.78790 to 0.83601, saving model to weights_best_double_lstm.hdf5
258/258 [==============================] - 380s 1s/step - loss: 0.5021 - accuracy: 0.8087 - val_loss: 0.4573 - val_accuracy: 0.8360
E

In [142]:
filepath="weights_best_double_lstm.hdf5"
model_dobule_lstm.load_weights(filepath)
evaluate_double_lstm = model_dobule_lstm.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(evaluate_double_lstm[0],evaluate_double_lstm[1]))

119/119 [==============================] - 22s 182ms/step - loss: 0.4506 - accuracy: 0.8573s -
Test set
  Loss: 0.451
  Accuracy: 0.857
